In [1]:
import prepare_proteins
import bsc_calculations
import os
import shutil

#### Crear carpeta para modelos PDB y renombrarlos

In [10]:
if not os.path.exists("structures"):
    os.mkdir("structures")
rank = 0 
for model in os.listdir("AF_structures/alphafold/output_models/"):
    if os.path.exists("AF_structures/alphafold/output_models/"+model+"/ranked_"+str(rank)+".pdb"):
        shutil.copyfile("AF_structures/alphafold/output_models/"+model+"/ranked_"+str(rank)+".pdb", "structures/"+model+".pdb") #Al hacer esto, no puedo tener múltiples 
                                                                                                    #modelos de una misma proteina¿? Los sobreescribo¿?

#### Preparar las proteínas

In [2]:
#La clase proteinModels lee los PDB mediante BioPython como Bio.PDB.Structures. Se obtiene un atributo "structures" que se trata de un 
#diccionario donde --> Key = nombre modelo y Value = Bio.PDB.Object 
models = prepare_proteins.proteinModels("structures")

#Iteramos en el diccionario para obtener el nombre de las estructuras.
for model in models:
    print(models.structures[model])

<Structure id=GPX_Bacillus-subtilis>
<Structure id=GPX_Gossypium-hirsutum>
<Structure id=GPX_Lactococcus-lactis>
<Structure id=GPX_Neisseria-meningitidis>
<Structure id=GPX_Staphylococcus-aureus>


In [12]:
#La librería prepare_proteins tiene un metodo para eliminar elementos random terminales de las estructuras de AF usando un confidence score 
#almacenado en la columna donde generalmente se almacena el B-factor del archivo PDB.

models.removeTerminiByConfidenceScore(confidence_threshold=90)

In [16]:
#El metodo saveModels guarda los nuevos modelos en una carpeta la cual se especifica su nuevo nombre. 
models.saveModels("trimmed_models")
models = prepare_proteins.proteinModels('trimmed_models/')

/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2548
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2516
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2517
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2803
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2539
  warnings.warn(


### Alinear estructuras con una referencia 

In [17]:
# Se alinean las estructuras utilizando una de referencia, con tal de tener un marco estructural común. Se genera una carpeta nueva 
# llamada "aligned_models".

models.alignModelsToReferencePDB('trimmed_models/GPX_Bacillus-subtilis.pdb', 'aligned_models', chain_indexes=0)

# Las proteínas alineadas se "preparan" nuevamente aplicando la librería anterior.
models = prepare_proteins.proteinModels('aligned_models/')


/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2550
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2518
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2519
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2805
  warnings.warn(
/home/idg29/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2541
  warnings.warn(


### Optimización con Prepwizard

In [20]:
#Llamamos al método y generamos carpeta "prepwizard" donde guardar estructuras optimizadas.
jobs = models.setUpPrepwizardOptimization('prepwizard')

#Después de ejecutar el método, este devuelve los comandos que deben ejecutarse para realizar la optimización de Prepwizard en 
#una máquina equipada con la licencia de software de Schrödinger. 
#Los comandos pueden pasarse a la biblioteca bsc_calculations para generar los scripts que faciliten la ejecución.

bsc_calculations.local.parallel(jobs, cpus=min([40, len(jobs)]))

#Pasamos los comandos a la subclase local y definimos de antemano el número de CPUs que queremos usar, de modo que la biblioteca creará un script para 
#cada CPU que se utilizará. Los archivos de script se llaman commands_?; donde "?" representa un número entero que identifica el script individual. 
#El método también escribe un script llamado commands, que puede lanzar todos los scripts simultáneamente. 
#Entonces, para ejecutar las optimizaciones, necesitamos cargar la carpeta 'prepwizard' en un clúster con una licencia de Schrödinger y 
#todos los scripts de comandos.

#Para ejecutar los calculos, debemos lanzar "bash commands"

In [28]:
#Después de optimizar los modelos mediante PrepWizard, debemos descargarlos en la carpeta que contenga también este mismo notebook y cargarlos en la libreria. 

models.loadModelsFromPrepwizardFolder('prepwizard/')

#Después de cargar los modelos en la librería, hay que evaluarlos visualmente. Para ello, se guardan en la carpeta "prepwizard_models"

models.saveModels('prepwizard_models')

/home/ivan/anaconda3/envs/BSC/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Atom H defined twice in residue <Residue SER het=  resseq=10 icode= > at line 16.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(


## Docking Previo Con GLIDE

In [ ]:
#Previo PELE, se deben propner cuales serán las múltilpes conformaciones que tendrá el ligando a la hora de interactuar con la proteína. Para ello, 
#se realizan docking previos de menor precision mediante programa Glide. Este programa genera un "cuadricula" constituida por un cubículo interno que 
#contiene el ligando y delimita la region central en la que se posiciona y orienta el ligando y el espacio en el que se exploran las posiciones y 
#orientaciones de este. El cubículo externo define el espacio en el que Glide permite que el ligando se acerque y se mueva antes de entrar en la región 
#de docking detallada. Asegura que el docking se enfoque en el sitio de unión, pero con la suficiente libertad para que el ligando pueda acomodarse 
#antes de fijarse en su posición ideal.

### Selección aminoacidos centrales docking

In [29]:
#Previa elaboración del docking con Glide, al trabajar con proteinas con centros catalíticos semejantes pero con una distribución de los aa 
#diferente y por ende con índices PDB diferentes para los residuos de interés, se debe realizar un alineamiento múltiple de secuéncia (MSA)previo. 
#Así pues, dado uno o múltiples aa de interés, mediante el MSA somos capaces de determinar en cada una de las proteínas, que índice adopta los aa
#de interés dentro de ellas. 

#El alineamiento se realiza mediante el método calculateMSA(), basado en el programa mafft.

msa = models.calculateMSA()

#Para evaluar el MSA, se genera un output con el siguiente comando, de tal manera que el resultado puede ser analizado con programa externo como CHIMERA.

prepare_proteins.alignment.writeMsaToFastaFile(msa, 'msa.fasta')

In [30]:
#Obtener los aa conservados

conserved_index = models.getConservedMSAPositions(msa)   #De aquí obtenemos cada átomo conservado en forma de tupla (chain_index, residue_id, X).
for c in conserved_index:
    if c[1] == 'C':
        print(c)    

(35, 'C')
(64, 'C')


In [18]:
#Usamos el índice del MSA para obtener los índices del PDB para todas las proteínas cargadas en la librería

aa_index = models.getStructurePositionsFromMSAindexes(35) #Este número se cambia por el índice que toque
print(aa_index) #índice del aa de interés en cada modelo (proteína)

#Con los índices PDB obtenidos para cada proteína, definimos el centro del cubículo de Glide

{'GPX_Bacillus-subtilis': [35], 'GPX_Gossypium-hirsutum': [43], 'GPX_Lactococcus-lactis': [35], 'GPX_Neisseria-meningitidis': [35], 'GPX_Staphylococcus-aureus': [36]}


### Preparación del cubículo para Glide

In [59]:
#Se define el centro del cubículo como el aa de interés, descrito como GC (grid center). Generamos un diccionario con 3 elementos
# (chain_id, residue_id, atom_name). Se usa Bio.PDB.Structures para interar los modelos y recopilar toda esta información de forma segura. 

aa_center_atom = {} # Create dictionary to store the atom 3-element tuple for each model
for model in models: # Iterate the models inside the library
    for r in models.structures[model].get_residues(): # Iterate the residues for each Bio.PDB.Structure object
        if r.id[1] == aa_index[model][0]: # Check that the residue matches the defined index
            assert r.resname == 'CYS', f"Residuo en el modelo {model} y residuo {r.id[1]} no es CYS, sino {r.resname}" # Assert that the residue has the correct residue identity. Assert devuelve un booleano True si se cumple la condición
                                      # de lo contrario devuelve un "AssertError"
            aa_center_atom[model] = (r.get_parent().id, r.id[1], 'SG') # Store the corresponsing tuple.

In [61]:
print(aa_center_atom)

{'GPX_Bacillus-subtilis': ('A', 35, 'SG'), 'GPX_Gossypium-hirsutum': ('A', 43, 'SG'), 'GPX_Lactococcus-lactis': ('A', 35, 'SG'), 'GPX_Neisseria-meningitidis': ('A', 35, 'SG'), 'GPX_Staphylococcus-aureus': ('A', 36, 'SG')}


In [62]:
#Un vez descrito el centro del grid, se crea la carpeta para los resultados de Glide y los script para lanzar los calculos que permitan ubicar el 
#grid en la proteína.

jobs = models.setUpDockingGrid('grid', aa_center_atom) # Set grid calcualtion. El méotodo necesita el nombre de la carpeta donde almacenar los calculos 
                                                        # y, el diccionario con los centros.
bsc_calculations.local.parallel(jobs, cpus=min([40, len(jobs)])) # Write the scripts to run them locally.


sh: 2: run: not found


### Realizar Glide Docking

In [ ]:
#Una vez ubicado el cubículo en los diferentes modelos de las proteínas, es necesario disponer de un ligando del cual estudiar el docking.abs
#Para ello, mediante la aplicación online de "Maestro 2D sketcher" podemos diseñar el ligando y guardarlo como archiovo ".mae" en el directorio 
#de este proyecto. En caso de descargar el ligando desde PDB en dicho formato, se utiliza el siguiente método para convertir todos los 
#ligandos presentes en una carpeta determinada a ".mae"

models.convertLigandPDBtoMae('ligands') #"ligands" es la carpeta que contiene inicialmente los ligandos en formato PDB.

In [ ]:
#Con el cubículo establecido y los ligandos preparados, se puede iniciar el docking.

jobs = models.setUpGlideDocking('docking', 'grid', 'ligands', poses_per_lig=100) #poses_per_lig hace referencia al número de trayectorias de docking
bsc_calculations.local.parallel(jobs, cpus=min([40, len(jobs)]))

#Tras ejecutar y descargar los resultados del acoplamiento, pueden analizarse para extraer las mejores poses y introducirlas a PELE.

#Las mejores poses se seleccionaran mediante el "GlideScore". sin embargo, en muchas ocasiones, también podemos querer filtrar las poses por 
#diferentes distancias proteína-ligando que representen las mejores conformaciones según lo que pretendamos simular. 
#Para estudiar esta sesgunda opción, en primer lugar, construimos un diccionario que contiene las distancias de acoplamiento que queremos calcular 
#a partir de las conformaciones generadas por el cálculo de acoplamiento de Glide. Este diccionario (atom_pairs) es un diccionario doblemente 
#anidado que debe ir primero para todos los modelos de proteínas y luego para todos los ligandos que fueron acoplados.

In [ ]:
atom_pairs = {}
for model in models:
    atom_pairs[model] = {}
    for ligand in ["x"]: #Siendo "x" ¿carpeta con poses del ligando estudiadas en Glide?
        atom_pairs[model][ligand] = []
        atom_pairs[model][ligand].append(aa_center_atom[model], "x") #Siendo "x" un átomo del ligando del cual se estudia el docking
atom_pairs    

In [ ]:
#Ahora podemos utilizar este diccionario como entrada para nuestro análisis de acoplamiento:

models.analyseDocking('docking', atom_pairs=atom_pairs)

#Los datos del análisis de acoplamiento se almacenan como un Panda DF en el atributo .docking_data:

models.docking_data

In [ ]:
#Selección de las mejores poses en función de la distáncia

#Para facilitar la selección de poses según una distancia común, primero necesitamos agrupar las distancias bajo un nombre común (metric_distances) 
#en nuestro data frame. Podemos utilizar el método combineDockingDistancesIntoMetrics(), que reunirá todas las distancias en una lista y las 
#combinará tomando el mínimo de los valores de distancia. Para utilizar la función, necesitamos construir un diccionario triplemente 
#anidado que vaya de cada métrica, modelo y ligando y contenga como valores las listas (de cada modelo + ligando), que se combinarán bajo el 
#nombre métrico común

metric_distances = {} # Define the global dictionary
metric_distances['X'] = {} # Define the metric nested dictionary
for model in models:
    metric_distances['X'][model] = {} # Define the model nested dictionary
    for ligand in models.docking_ligands[model]: 
        # Define the ligand nested dictionary with all the docking distances list
        metric_distances['X'][model][ligand] = models.getDockingDistances(model, ligand)

In [ ]:
#Ahora le damos este diccionario al método combineDockingDistancesIntoMetrics() e inspeccionamos los cambios sobre nuestros datos de acoplamiento:

models.combineDockingDistancesIntoMetrics(metric_distances)
models.docking_data

#Ahora que nuestro marco de datos de acoplamiento contiene una nueva columna con las distancias de acoplamiento agrupadas bajo el mismo nombre, 
#podemos utilizar esta información para extraer las mejores poses resultantes de cada ejecución de acoplamiento.

### Cálculo de las mejores poses

In [ ]:
#Esto puede conseguirse dando un valor umbral para obtener todas las poses catalíticas que cumplan un valor específico para la métrica común y 
#obteniendo las que tengan la mejor puntuación Glide. Sin embargo, no siempre ocurre que todas las ejecuciones de docking (es decir, para cada 
#combinación de proteína y ligando) produzcan distancias razonables. En este caso, muchos resultados de docking quedarían fuera si los umbrales de 
#distancia empleados fueran demasiado restrictivos. Una forma de solventar esto sería seleccionar un umbral de distancia pequeño, elegir los mejores 
#modelos que lo cumplieran y, a continuación, aumentar este umbral en una pequeña cantidad para reunir los mejores modelos que contengan distancias 
#métricas ligeramente grandes. Si repetimos esto de forma iterativa, podríamos obtener los mejores modelos posibles para todas las ejecuciones de 
#acoplamiento sin comprometer nuestra elección utilizando un umbral mayor para poses con valores métricos ya buenos.

#Por suerte, este proceso se ha automatizado en la biblioteca para que la selección de acoplamiento pueda llevarse a cabo fácilmente:

best_poses = models.getBestDockingPosesIteratively(metric_distances, max_threshold=5.0)
best_poses

#Para usar el método, tan solo debemos aportarle el diccionario con las métricas. El método define el proceso de iteración en base a los 
#siguientes keywords: min_threshold = 3.5, max_threshold = 5.0 y step_size = 0.1

#Las mejores poses así seleccionadas se devuelven como  pandas DF y se utilizan para configurar el cálculo final de PELE.

### Extracción mejores poses

In [ ]:
#Podemos extraer un subconjunto de poses de docking dando un pandas DF filtrado al método extractDockingPoses(). 
#En nuestro caso, usamos el marco de datos 'best_poses' que contiene una pose por cada ejecución de docking.

models.extractDockingPoses(best_poses, 'docking', 'best_poses', separator='@')

#Las entradas son el marco de datos filtrado, la carpeta 'docking' y la carpeta donde almacenar las poses. Los nombres de los archivos se 
#darán como una combinación entre model_name+ligand+docking_pose y separados con el símbolo proporcionado en la palabra clave 'separator'. 
#Se emitirá una advertencia si el nombre de la proteína o del ligando contiene el separador en su nombre.

### PELE

In [ ]:
#Una vez extraídas las poses de acoplamiento, el último paso es configurar las carpetas de cálculo PELE, los archivos y los scripts slurm. 
#El método necesita la carpeta donde almacenar la información de cálculo ('pele'), la carpeta que contiene las poses de docking ('best_poses'), 
#y un archivo YAML de entrada que contenga los detalles del protocolo de la plataforma PELE. También damos a nuestro diccionario atom_pairs para 
#calcular las mismas distancias empleadas en nuestro análisis de docking, por lo que se calculan a lo largo de la simulación PELE.

#Dado que el índice de ligando en nuestras poses de docking es cero, cambiamos el índice de ligando por defecto (1) con la palabra clave 
#ligand_index. Por último, como cambiamos el separador al escribir las poses de docking, tenemos que hacer que el método setUpPELECalculation() 
#sea consciente utilizando de nuevo la palabra clave 'separator'.

jobs = models.setUpPELECalculation('pele', 'best_poses/', 'input.yaml', distances=atom_pairs, ligand_index=0, separator='@')

#Los comandos generados por este método son ligeramente diferentes a los de otras funciones. Sin embargo, pueden procesarse rápidamente empleando ç
#el método setUpPELEForMarenostrum() dentro de la biblioteca bsc_calculations.

bsc_calculations.marenostrum.setUpPELEForMarenostrum(jobs)